In [ ]:
import base64
from pdf2image import convert_from_path
from PIL import Image,ImageFile
import io
import pytesseract
import os
import pandas as pd
import csv
import sys
pytesseract.pytesseract.tesseract_cmd = r"D:\Tesseract-OCR\tesseract.exe"
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
import cv2
import numpy as np
import time

def text_extractor(img_pil,lng='eng',i=0):
    if img_pil.size[0]>700 or img_pil.size[1]>700:
        size=700
        wpercent = (size / float(img_pil.size[0]))
        hsize = int((float(img_pil.size[1]) * float(wpercent)))
        img_pil = img_pil.resize((size, hsize), Image.ANTIALIAS)
    img=np.array(img_pil)
    try:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except cv2.error:
        pass
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_AREA)
    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    result = pytesseract.image_to_string(img,lang=lng)
    if len(result)==0 and i==0:
        img_pil=img_pil.transpose(Image.ROTATE_270)
        result=text_extractor(img_pil,lng,i=1)
    if len(result)==0 and i==1:
        img_pil=img_pil.transpose(Image.ROTATE_90)
        result=text_extractor(img_pil,lng,i=2)
    return result
def pdf_converter(pdf):
    pdf=base64.b64decode(pdf)
    file = open('new.pdf', 'w+b')
    file.write(pdf)
    file.close()
    pages = convert_from_path(r'new.pdf')
    return pages[0]

result=pd.DataFrame(columns=['Requestid','id','format','time','data_rus','data_eng'])
path='d:\Python\cv\Фото'

csv.field_size_limit(100000000)
for file in ['09_2018.csv','10_2018.csv','11_2018.csv','12_2018.csv','01_2019.csv']:
    data=pd.read_csv(os.path.join(path,file), engine='python',sep=';',encoding = "utf-8",header=None,names=['id','requestid','Documenttype','Documentname','b64'])
    data=data[~data['b64'].isnull()].reset_index(drop=True)
    for i in range(len(data)):
        start_time = time.time()
        formt=data.Documentname[i].split('.')[-1]
        try:
            if formt=='pdf' or formt=='PDF':
                img=pdf_converter(data.b64[i])
            else:
                img=Image.open(io.BytesIO(base64.b64decode(data.b64[i])))
            txt_rus=text_extractor(img,lng='rus')
            txt_eng=text_extractor(img)
        except Exception:
            continue
        end_time=time.time()
        result.loc[len(result)]=[data.requestid[i],data.id[i],formt,(end_time - start_time),txt_rus,txt_eng]
result

In [69]:
%%time
import base64
from pdf2image import convert_from_path
from PIL import Image,ImageFile
import io
import os
import pandas as pd
import csv
import sys
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
import cv2
import numpy as np
import time
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import preprocess_input
from keras.applications.nasnet import decode_predictions
from keras.preprocessing import image
model=NASNetLarge()
def preprocess_image(img_data):
    size=331
#    wpercent = (size / float(img_pil.size[0]))
#    hsize = int((float(img_pil.size[1]) * float(wpercent)))
    img_pil = img_data.resize((size, size), Image.ANTIALIAS)
    img_pil=image.img_to_array(img_pil)
    img_pil=np.expand_dims(img_pil,axis=0)
    img_pil=preprocess_input(img_pil)
    return img_pil
def class_prediction(img_pil,model=model):
    features=model.predict(img)
    result=decode_predictions(features)
    result=[result[0][0][1],result[0][1][1],result[0][2][1]]
    return result
def pdf_converter(pdf):
    pdf=base64.b64decode(pdf)
    file = open('new.pdf', 'w+b')
    file.write(pdf)
    file.close()
    pages = convert_from_path(r'new.pdf')
    return pages[0]

result=pd.DataFrame(columns=['Requestid','id','format','time','category_1','category_2','category_3'])
path='d:\Python\cv\Фото'
csv.field_size_limit(100000000)
for file in os.listdir(path):
    data=pd.read_csv(os.path.join(path,file), engine='python',sep=';',encoding = "utf-8",header=None,names=['id','requestid','Documenttype','Documentname','b64'])
    data=data[~data['b64'].isnull()].reset_index(drop=True)
    for i in range(len(data)):
        start_time = time.time()
        formt=data.Documentname[i].split('.')[-1]
        try:
            if formt=='pdf' or formt=='PDF':
                img=pdf_converter(data.b64[i])
            else:
                img=Image.open(io.BytesIO(base64.b64decode(data.b64[i]))).convert('RGB')
            img=preprocess_image(img)
            temp_data=class_prediction(img)
        except Exception:
            continue
        end_time=time.time()
        result.loc[len(result)]=[data.requestid[i],data.id[i],formt,(end_time - start_time),temp_data[0],temp_data[1],temp_data[2]]
result

Wall time: 1h 23min 22s


In [72]:
result=pd.read_csv('photo_category.csv')
result2=result.loc[(result['category_1']!='web_site')&(result['category_1']!='slide_rule')&
          (result['category_2']!='web_site')&(result['category_2']!='slide_rule')&
          (result['category_3']!='web_site')&(result['category_3']!='slide_rule')].sort_values(by=['Requestid'])
alldata=alldata.loc[~alldata['requestid'].isin(result2['Requestid'].values)]
alldata.to_csv('right_data.csv')

In [13]:
import os
import pandas as pd
import csv
import sys
path='d:\Python\cv\Фото'
csv.field_size_limit(100000000)
for num,file in enumerate(os.listdir(path)):
    data=pd.read_csv(os.path.join(path,file), engine='python',sep=';',encoding = "utf-8",header=None,names=['id','requestid','Documenttype','Documentname','b64'])
    data=data[~data['b64'].isnull()].reset_index(drop=True)
    if num==0:
        alldata=data
    else:
        alldata=alldata.append(data)
result=pd.read_csv('photo_category.csv')
result2=result.loc[(result['category_1']!='web_site')&(result['category_1']!='slide_rule')&
          (result['category_2']!='web_site')&(result['category_2']!='slide_rule')&
          (result['category_3']!='web_site')&(result['category_3']!='slide_rule')].sort_values(by=['Requestid'])
alldata=alldata.loc[alldata['requestid'].isin(result['Requestid'].values)]
alldata.to_csv('not_right_data.csv')


In [17]:
import os
import pandas as pd
import csv
import sys
from keras.applications.nasnet import preprocess_input
import base64
import io
from pdf2image import convert_from_path
from PIL import Image,ImageFile
data=pd.read_csv('not_right_data.csv')
data.requestid=data.requestid.astype(int)
path='d:\Python\cv\\test_data'
def preprocess_image(img_data):
    size=331
#    wpercent = (size / float(img_pil.size[0]))
#    hsize = int((float(img_pil.size[1]) * float(wpercent)))
    img_pil = img_data.resize((size, size), Image.ANTIALIAS)
    return img_pil
def pdf_converter(pdf):
    pdf=base64.b64decode(pdf)
    file = open('new.pdf', 'w+b')
    file.write(pdf)
    file.close()
    pages = convert_from_path(r'new.pdf')
    return pages[0]
for i in range(len(data)):
    formt=data.Documentname[i].split('.')[-1]
    try:
        if formt=='pdf' or formt=='PDF':
            img=pdf_converter(data.b64[i])
        else:
            img=Image.open(io.BytesIO(base64.b64decode(data.b64[i]))).convert('RGB')
        img=preprocess_image(img)
    except Exception:
        continue
    img.save(os.path.join(path,str(data.requestid[i])+'_'+str(data.id[i])+'.jpg'))

In [1]:
import os
import pandas as pd
import csv
import sys
from keras.applications.nasnet import preprocess_input
import base64
import io
from pdf2image import convert_from_path
from PIL import Image,ImageFile
csv.field_size_limit(100000000)
data=pd.read_csv(r'D:\Python\cv\\CBU\val_bad.csv', engine='python',sep=';',encoding = "utf-8",skiprows=1,header=None,names=['id','requestid','Documenttype','Documentname','b64'], error_bad_lines=False)
#data.requestid=data.requestid.astype(int)
path='d:\Python\cv\\dop_bad'
def preprocess_image(img_data):
    size=224
#    wpercent = (size / float(img_pil.size[0]))
#    hsize = int((float(img_pil.size[1]) * float(wpercent)))
    img_pil = img_data.resize((size, size), Image.ANTIALIAS)
    return img_pil
def pdf_converter(pdf):
    pdf=base64.b64decode(pdf)
    file = open('new.pdf', 'w+b')
    file.write(pdf)
    file.close()
    pages = convert_from_path(r'new.pdf')
    return pages[0]
for i in range(len(data)):
    formt=data.Documentname[i].split('.')[-1]
    try:
        if formt=='pdf' or formt=='PDF':
            img=pdf_converter(data.b64[i])
        else:
            img=Image.open(io.BytesIO(base64.b64decode(data.b64[i]))).convert('RGB')
#        img=preprocess_image(img)
    except Exception:
        continue
    img.save(os.path.join(path,str(data.requestid[i])+'_'+str(data.id[i])+'.jpg'))

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%%time
import base64
from pdf2image import convert_from_path
from PIL import Image,ImageFile
import io
import os
import pandas as pd
import csv
import sys
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
import cv2
import numpy as np
import time

result=pd.DataFrame(columns=['Requestid','id','format','size1','size2','size3'])
path='d:\Python\cv\Фото'
csv.field_size_limit(100000000)
for file in os.listdir(path):
    data=pd.read_csv(os.path.join(path,file), engine='python',sep=';',encoding = "utf-8",header=None,names=['id','requestid','Documenttype','Documentname','b64'])
    data=data[~data['b64'].isnull()].reset_index(drop=True)
    for i in range(len(data)):
        start_time = time.time()
        formt=data.Documentname[i].split('.')[-1]
        try:
            if formt=='pdf' or formt=='PDF':
                img=pdf_converter(data.b64[i])
            else:
                img=Image.open(io.BytesIO(base64.b64decode(data.b64[i]))).convert('RGB')
            img=image.img_to_array(img)
        except Exception:
            continue
        end_time=time.time()
        result.loc[len(result)]=[data.requestid[i],data.id[i],formt,img.shape[0],img.shape[1],img.shape[2]]
result

Wall time: 11min 2s


In [5]:
import os
import torch,torchvision 
device='cpu'
model=torch.load('d:\Python\\cv\\pytorch_model\\second_version\\pytorch_model_train-1.000_test-0.984.pt')
model.eval()
path='D:\Python\cv\TEST'
test_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize(224),
                                      torchvision.transforms.ToTensor(),
                                     ])
def predict_image(image):
    image=test_transforms(image).float()
    image=image.unsqueeze_(0)
    image=image.to(device)
    output=model(image)
    return output.data.cpu().numpy().argmax()
from PIL import Image
for file in os.listdir(path):
    im=Image.open(os.path.join(path,file))
    if predict_image(im)==0:
        im.save("d:\Python\cv\\1\\bad\\"+file)
    else:
        im.save("d:\Python\cv\\1\\good\\"+file)

In [1]:
from pdf2image import convert_from_path
im=convert_from_path(r'D:\Python\cv\zapekin_spravka.pdf',strict=True)[0]
device='cpu'
import torch,torchvision
model=torch.load(r"C:\Users\\Toshevikov\\IPython\\Production\\pytorch_model_train-1.000_test-0.984.pt")
model.eval()
test_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize(224),
                                      torchvision.transforms.ToTensor(),
                                     ])
#функция прогноза модели
def predict_image(image):
    image=test_transforms(image).float()
    image=image.unsqueeze_(0)
    image=image.to(device)
    output=model(image)
    return output.data.cpu().numpy().argmax()
#predict_image(im)
import numpy as np
#np.array(im)
im

KeyboardInterrupt: 